In [1]:
import numpy as np
import pandas as pd 

In [2]:
tab1 = pd.read_csv("./data/table1.csv")
tab1.rename(columns={
    "شناسه‌ی عضو" : 'personID',
    "شناسه‌ی بخش" : 'unitID',
    "ضریب اهمیت در بخش":"importance_unit",
    "میزان انتظار":"expected"
},inplace=True)
tab1.head()

,personID,unitID,importance_unit,expected
0,810,1,0.82,21
1,284,1,0.73,34
2,175,1,0.83,43
3,436,1,0.85,44
4,289,1,0.71,31


In [3]:
tab1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1484 entries, 0 to 1483
Data columns (total 4 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   personID         1484 non-null   int64  
 1   unitID           1484 non-null   int64  
 2   importance_unit  1484 non-null   float64
 3   expected         1484 non-null   int64  
dtypes: float64(1), int64(3)
memory usage: 46.5 KB


In [4]:
tab2  = pd.read_csv("./data/table2.csv")
tab2.rename(columns={
  'شناسه‌ی بخش': 'unitID',
  'حقوق ساعتی':'hourly_income'
},inplace=True)
tab2.head()

,unitID,hourly_income
0,1,36000
1,2,26000
2,3,20000
3,4,24000
4,5,35000


In [5]:
tab2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10 entries, 0 to 9
Data columns (total 2 columns):
 #   Column         Non-Null Count  Dtype
---  ------         --------------  -----
 0   unitID         10 non-null     int64
 1   hourly_income  10 non-null     int64
dtypes: int64(2)
memory usage: 288.0 bytes


In [6]:
tab3 = pd.read_csv("./data/table3.csv")
tab3.rename(columns={
    'شناسه‌ی عضو':"personID",
    'شناسه‌ی بخش':'unitID',
    'ماه':'month',
    'ساعت کاری':'working_time'
},inplace=True)
tab3.head()

,personID,unitID,month,working_time
0,810,1,1,20
1,810,1,2,13
2,810,1,3,23
3,810,1,4,25
4,810,1,5,14


In [7]:
tab3.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17808 entries, 0 to 17807
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype
---  ------        --------------  -----
 0   personID      17808 non-null  int64
 1   unitID        17808 non-null  int64
 2   month         17808 non-null  int64
 3   working_time  17808 non-null  int64
dtypes: int64(4)
memory usage: 556.6 KB


In [8]:
months = pd.DataFrame({"month": range(1, 13)})
grid = (
    tab1[["personID", "unitID", "expected"]]
    .merge(months, how="cross")  
)
grid.head()

,personID,unitID,expected,month
0,810,1,21,1
1,810,1,21,2
2,810,1,21,3
3,810,1,21,4
4,810,1,21,5


In [9]:
work = tab3[["personID", "unitID", "month", "working_time"]]
work

,personID,unitID,month,working_time
0,810,1,1,20
1,810,1,2,13
2,810,1,3,23
3,810,1,4,25
4,810,1,5,14
...,...,...,...,...
17803,578,10,8,55
17804,578,10,9,34
17805,578,10,10,21
17806,578,10,11,60


In [10]:
grid = grid.merge(work,on=["personID","unitID","month"],how='left')
grid

,personID,unitID,expected,month,working_time
0,810,1,21,1,20
1,810,1,21,2,13
2,810,1,21,3,23
3,810,1,21,4,25
4,810,1,21,5,14
...,...,...,...,...,...
17803,578,10,37,8,55
17804,578,10,37,9,34
17805,578,10,37,10,21
17806,578,10,37,11,60


In [11]:
grid.isna().sum()

personID        0
unitID          0
expected        0
month           0
working_time    0
dtype: int64

In [12]:
grid["below_expect_in_unit"] = grid["working_time"] < grid["expected"]

grid

,personID,unitID,expected,month,working_time,below_expect_in_unit
0,810,1,21,1,20,True
1,810,1,21,2,13,True
2,810,1,21,3,23,False
3,810,1,21,4,25,False
4,810,1,21,5,14,True
...,...,...,...,...,...,...
17803,578,10,37,8,55,False
17804,578,10,37,9,34,True
17805,578,10,37,10,21,True
17806,578,10,37,11,60,False


In [13]:
lost_by_member_month = (
    grid.groupby(["personID", "month"], as_index=False)["below_expect_in_unit"]
        .any()
        .rename(columns={"below_expect_in_unit": "lost_this_month"})
)
lost_by_member_month.iloc[:14,:]

,personID,month,lost_this_month
0,1,1,True
1,1,2,True
2,1,3,True
3,1,4,True
4,1,5,False
5,1,6,True
6,1,7,True
7,1,8,True
8,1,9,True
9,1,10,True


In [14]:
lost_count = (
    lost_by_member_month.groupby("personID", as_index=False)["lost_this_month"]
        .sum()
        .rename(columns={"lost_this_month": "lost_months"})
)
lost_count

,personID,lost_months
0,1,10
1,2,10
2,3,10
3,4,6
4,5,10
...,...,...
796,994,5
797,996,11
798,997,7
799,998,9


In [15]:
fired = lost_count.assign(fired=lambda d: d["lost_months"] >= 6)
fired

,personID,lost_months,fired
0,1,10,True
1,2,10,True
2,3,10,True
3,4,6,True
4,5,10,True
...,...,...,...
796,994,5,False
797,996,11,True
798,997,7,True
799,998,9,True


In [16]:
fired_list = fired.loc[fired["fired"], "personID"].tolist()
survivors_list = fired.loc[~fired["fired"], "personID"].tolist()